## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient

from duration_notebook import categorical, y_pred
from sklearn.metrics import mean_squared_error

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

experiments = client.search_experiments()
for exp in experiments:
    print(exp.experiment_id, exp.name, exp.artifact_location)

2025/07/10 08:43:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/10 08:43:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


1 nyc_taxi_trip_duration_experiment /Users/jordanharris/Code/mlops_zoomcamp/mlruns/1
0 Default mlflow-artifacts:/0


In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f58671d315a64af2b4d319d2a00e86e0, rmse: 6.3074
run id: a5c440126bb64bc4ade324f5f2640eba, rmse: 6.3080
run id: cf8131686fa14689a51c35c2515985e6, rmse: 6.3113
run id: a765a12581a845b1857a7ce3aff63c5c, rmse: 6.3115
run id: eb8bbe100b3e42da9318f335c55e9ac2, rmse: 6.3140


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
#run_id = "b8904012c84343b5bf8ee72aa8f0f402"
my_run_id = "096abcbedcbe4395a436ff10d8a7a28e"
model_uri = f"runs:/{my_run_id}/xgboost_model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/07/10 10:45:38 WARNING mlflow.tracking._model_registry.fluent: Run with id 096abcbedcbe4395a436ff10d8a7a28e has no artifacts at artifact path 'xgboost_model', registering model based on models:/m-e42b486c498f4f5db7b84785c8104a26 instead
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1752137138148, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1752137138148, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='096abcbedcbe4395a436ff10d8a7a28e', run_link=None, source='models:/m-e42b486c498f4f5db7b84785c8104a26', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [13]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_72545/3243314508.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [17]:
model_version = 1
new_stage = "Staging"
#client.transition_model_version_stage(
#    name=model_name,
#    version=model_version,
#    stage=new_stage,
#    archive_existing_versions=False
#)


client.set_registered_model_alias(name=model_name, alias=new_stage, version=model_version)


In [18]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['Staging'], creation_timestamp=1752137138148, current_stage='None', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-07-10', last_updated_timestamp=1752138686452, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='096abcbedcbe4395a436ff10d8a7a28e', run_link=None, source='models:/m-e42b486c498f4f5db7b84785c8104a26', status='READY', status_message=None, tags={}, user_id=None, version=1>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [50]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [46]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = pd.to_datetime(df.lpep_dropoff_datetime) - pd.to_datetime(df.lpep_pickup_datetime)
    df.duration = df['duration'].apply(lambda x: x.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    #numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return {"rsme": np.sqrt(mse)}

In [23]:
#df = read_dataframe("data/green_tripdata_2021-03.csv")
df = pd.read_parquet("data/green_tripdata_2021-03.parquet")


In [25]:
client.download_artifacts(run_id=my_run_id, path='preprocessor', dst_path='module_2')

'/Users/jordanharris/Code/mlops_zoomcamp/module_2/preprocessor'

In [29]:
import pickle

with open("models/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [32]:
X_test = preprocess(df, dv)

In [35]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
df['duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60

target = "duration"
y_test = df[target].values

In [54]:
client.set_registered_model_alias(
    name="nyc-taxi-regressor",
    alias="production",
    version=1
)

In [51]:
%time test_model(name=model_name, alias="staging", X_test=X_test, y_test=y_test)

CPU times: user 14.8 s, sys: 177 ms, total: 15 s
Wall time: 2.46 s


{'rsme': np.float64(54.10256898844454)}

In [56]:
%time test_model(name=model_name, alias="production", X_test=X_test, y_test=y_test)

CPU times: user 14.5 s, sys: 197 ms, total: 14.7 s
Wall time: 2.65 s


{'rsme': np.float64(54.10256898844454)}

In [57]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_72545/1470871855.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['production', 'staging'], creation_timestamp=1752137138148, current_stage='Production', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-07-10', last_updated_timestamp=1752140265351, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='096abcbedcbe4395a436ff10d8a7a28e', run_link=None, source='models:/m-e42b486c498f4f5db7b84785c8104a26', status='READY', status_message=None, tags={}, user_id=None, version=1>